**Note, if you had opened the notebook before doing the pip installs in the setup notebook, now it's a good moment to restart the notebook's kernel**

# Set db credentials

for the MCP servers to talk to Teradata, we need to give them the db credentials

In [1]:
host = 'host.docker.internal'
user = 'demo_user'

In [2]:
import getpass
db_pw = getpass.getpass("Enter DB Password here")

Enter DB Password here ················


# Start various MCP Servers

In [3]:
import os

In [4]:
import chatbot_widget

In [5]:
from chatbot_widget.mcp.server_manager import MCPServerManager
mcp = MCPServerManager()

## List available Servers (Example Servers)

In [7]:
from pathlib import Path

available_servers = [
    str(Path(p))
    for p in mcp.list_example_servers()
]
available_servers


['/home/jovyan/.local/lib/python3.11/site-packages/chatbot_widget/mcp/example_server/ascii_server.py',
 '/home/jovyan/.local/lib/python3.11/site-packages/chatbot_widget/mcp/example_server/random_server.py',
 '/home/jovyan/.local/lib/python3.11/site-packages/chatbot_widget/mcp/example_server/teradatasql_server.py']

the file path for the full teradata mcp server is this:

you can use `!cat <path> to see the source code`

## inspect source code

In [8]:
from IPython.display import Markdown

ascii_path = available_servers[0]
source_info = mcp.show_server_source(script_path=str(ascii_path))
print(source_info["source"])


   1: from fastmcp import FastMCP
   2: import pyfiglet
   3: import sys
   4: 
   5: mcp = FastMCP("ASCII Art MCP Server")
   6: 
   7: @mcp.tool("ascii_art")
   8: def ascii_art(text: str):
   9:     """Generate simple ASCII art for a short string (max 10 chars)."""
  10:     if not text:
  11:         raise ValueError("Text must not be empty.")
  12:     if len(text) > 10:
  13:         raise ValueError("Text too long (max 10 characters).")
  14:     return pyfiglet.figlet_format(text, font="standard")
  15: 
  16: @mcp.tool("mirror")
  17: def mirror_text(text: str):
  18:     """Return text mirrored (reversed)."""
  19:     if len(text) > 20:
  20:         raise ValueError("Text too long (max 20 characters).")
  21:     return text[::-1]
  22: 
  23: if __name__ == "__main__":
  24:     port = 8770  # default port, override via CLI
  25:     if len(sys.argv) > 1:
  26:         port = int(sys.argv[1])
  27:     mcp.run(transport="http", port=port)


# Play with example server

## start the server

In [9]:
mcp.start(
    name = "randoms",
    script_path= '/home/jovyan/.local/lib/python3.11/site-packages/chatbot_widget/mcp/example_server/random_server.py',
    port=None  #8765 # none lets the server manager automatically choose a free port
    ,seed= 42 
)

{'status': 'ok',
 'message': "Started 'randoms' on port 8749 (PID=1447)",
 'healthy': True,
 'port': 8749,
 'log_file': 'randoms_server.log'}

## check health

In [10]:
mcp.check_health("randoms")

True

## list all tools

In [11]:
mcp.list_tools("randoms")

[Tool(name='numbers', title=None, description='Generate a list of random integers.', inputSchema={'properties': {'count': {'default': 5, 'type': 'integer'}, 'min_value': {'default': 0, 'type': 'integer'}, 'max_value': {'default': 100, 'type': 'integer'}}, 'type': 'object'}, outputSchema={'properties': {'result': {'items': {'type': 'integer'}, 'type': 'array'}}, 'required': ['result'], 'type': 'object', 'x-fastmcp-wrap-result': True}, icons=None, annotations=None, meta={'_fastmcp': {'tags': []}}),
 Tool(name='greet', title=None, description='Returns a friendly greeting.', inputSchema={'properties': {}, 'type': 'object'}, outputSchema={'properties': {'result': {'type': 'string'}}, 'required': ['result'], 'type': 'object', 'x-fastmcp-wrap-result': True}, icons=None, annotations=None, meta={'_fastmcp': {'tags': []}})]

## test a tool

In [12]:
mcp.test_tool("randoms","numbers",dict(count=3, min_value=1, max_value = 6 ) )

{'status': 'ok',
 'result': CallToolResult(content=[TextContent(type='text', text='[6,1,6]', annotations=None, meta=None)], structured_content={'result': [6, 1, 6]}, data=[6, 1, 6], is_error=False)}

# Test the Teradata MCP Server Tools

In [19]:
mcp.inspect_cli_arguments(script_path="tdmcpcommunity.py")

{'status': 'ok',
 'path': '/home/jovyan/JupyterLabRoot/mcp_server_workbench/tdmcpcommunity.py',
 'arguments': [{'flags': ['port'],
   'options': {'type': 'int', 'help': "'HTTP port (server listens here)'"}},
  {'flags': ['--host'],
   'options': {'required': 'True', 'help': "'Teradata DB host'"}},
  {'flags': ['--user'],
   'options': {'required': 'True', 'help': "'Teradata DB user'"}},
  {'flags': ['--password'],
   'options': {'required': 'True', 'help': "'Teradata DB password'"}}]}

In [13]:
mcp.start(
    "tdmcpcommunity",
    os.path.abspath("tdmcpcommunity.py"),
    port=None,  # or fixed port like 8765
    host=host,
    user=user,
    password=db_pw
)


{'status': 'warning',
 'message': "Started 'tdmcpcommunity' on port 8882 (PID=1455)",
 'healthy': False,
 'port': 8882,
 'log_file': 'tdmcpcommunity_server.log'}

In [15]:
#wait for 5 seconds
mcp.check_health("tdmcpcommunity")

True

In [16]:
mcp.list_tools("tdmcpcommunity")

[Tool(name='sec_rolePermissions', title=None, description='\n    Get permissions for a role.\n\n    Arguments:\n      role_name - role name to analyze\n\n    Returns:\n      ResponseType: formatted response with query results + metadata\n    ', inputSchema={'properties': {'role_name': {'type': 'string'}}, 'required': ['role_name'], 'type': 'object'}, outputSchema=None, icons=None, annotations=None, meta={'_fastmcp': {'tags': []}}),
 Tool(name='sec_userDbPermissions', title=None, description='\n    Get permissions for a user.\n\n    Arguments:\n      user_name - user name to analyze\n\n    Returns:\n      ResponseType: formatted response with query results + metadata\n    ', inputSchema={'properties': {'user_name': {'type': 'string'}}, 'required': ['user_name'], 'type': 'object'}, outputSchema=None, icons=None, annotations=None, meta={'_fastmcp': {'tags': []}}),
 Tool(name='sec_userRoles', title=None, description='\n    Get roles assigned to a user.\n\n    Arguments:\n      user_name - 

In [20]:
mcp.test_tool("tdmcpcommunity","dba_databaseVersion")

{'status': 'ok',
 'result': CallToolResult(content=[TextContent(type='text', text='{\n  "status": "success",\n  "results": [\n    {\n      "InfoKey": "VERSION",\n      "InfoData": "20.00.24.60"\n    },\n    {\n      "InfoKey": "LANGUAGE SUPPORT MODE",\n      "InfoData": "Standard"\n    },\n    {\n      "InfoKey": "RELEASE",\n      "InfoData": "20.00.24.60"\n    }\n  ],\n  "metadata": {\n    "tool_name": "base_readQuery",\n    "sql": "select InfoKey, InfoData FROM DBC.DBCInfoV;\\n",\n    "columns": [\n      {\n        "name": "InfoKey",\n        "type": "str"\n      },\n      {\n        "name": "InfoData",\n        "type": "str"\n      }\n    ],\n    "row_count": 3\n  }\n}', annotations=None, meta=None)], structured_content=None, data=None, is_error=False)}

In [21]:
mcp.test_tool("tdmcpcommunity","base_tableList", {"database_name":user})

{'status': 'ok',
 'result': CallToolResult(content=[TextContent(type='text', text='{\n  "status": "success",\n  "results": [\n    {\n      "TableName": "order_payments"\n    },\n    {\n      "TableName": "dim_payment_method"\n    },\n    {\n      "TableName": "key_customer"\n    },\n    {\n      "TableName": "lim_payments"\n    },\n    {\n      "TableName": "customer"\n    },\n    {\n      "TableName": "fct_orders"\n    },\n    {\n      "TableName": "sim_orders"\n    },\n    {\n      "TableName": "raw_customers"\n    },\n    {\n      "TableName": "sim_payments"\n    },\n    {\n      "TableName": "stg_customers"\n    },\n    {\n      "TableName": "dim_customers"\n    },\n    {\n      "TableName": "lim_orders"\n    },\n    {\n      "TableName": "lim_customers"\n    },\n    {\n      "TableName": "sim_customers"\n    },\n    {\n      "TableName": "raw_payments"\n    },\n    {\n      "TableName": "stg_payments"\n    },\n    {\n      "TableName": "stg_orders"\n    },\n    {\n      "TableName

# check health of all running servers

In [22]:
mcp.check_all()

{'randoms': True, 'tdmcpcommunity': True}

# Inspect all server's tools

In [23]:
mcp.get_tool_server_dict()

{'numbers': 'randoms',
 'greet': 'randoms',
 'sec_rolePermissions': 'tdmcpcommunity',
 'sec_userDbPermissions': 'tdmcpcommunity',
 'sec_userRoles': 'tdmcpcommunity',
 'dba_databaseSpace': 'tdmcpcommunity',
 'dba_resusageSummary': 'tdmcpcommunity',
 'dba_tableSpace': 'tdmcpcommunity',
 'dba_tableSqlList': 'tdmcpcommunity',
 'dba_tableUsageImpact': 'tdmcpcommunity',
 'dba_userSqlList': 'tdmcpcommunity',
 'tmpl_nameOfTool': 'tdmcpcommunity',
 'plot_line_chart': 'tdmcpcommunity',
 'plot_pie_chart': 'tdmcpcommunity',
 'plot_polar_chart': 'tdmcpcommunity',
 'plot_radar_chart': 'tdmcpcommunity',
 'sql_Analyze_Cluster_Stats': 'tdmcpcommunity',
 'sql_Execute_Full_Pipeline': 'tdmcpcommunity',
 'sql_Retrieve_Cluster_Queries': 'tdmcpcommunity',
 'base_columnDescription': 'tdmcpcommunity',
 'base_databaseList': 'tdmcpcommunity',
 'base_readQuery': 'tdmcpcommunity',
 'base_tableAffinity': 'tdmcpcommunity',
 'base_tableDDL': 'tdmcpcommunity',
 'base_tableList': 'tdmcpcommunity',
 'base_tablePreview':

# stop all Servers

In [24]:
mcp.stop_all()

[{'status': 'ok', 'message': "Stopped 'randoms' (port=8749, PID=1447)."},
 {'status': 'ok',
  'message': "Stopped 'tdmcpcommunity' (port=8882, PID=1455)."}]

In [25]:
# run again to verify
mcp.stop_all()

{'status': 'ok', 'message': 'No servers running.'}

In [26]:
#should return error
mcp.test_tool("tdmcpcommunity","dba_databaseSpace", {"database_name":user})

{'status': 'error', 'message': "Server 'tdmcpcommunity' not running."}